In [19]:
from IPython.display import display
import spotipy
import spotipy.util as util
import pickle

client_id = 'b69a9985fa8842deb0691b2d0e3f0b69'
client_secret = 'd9e9ae2924174c139a5a9ccb303f9f3a'
redirect_uri = 'http://localhost/'

username = '22mrmbu7oumkrb56tcsclawdi'

token = util.prompt_for_user_token(username, 'user-library-read', client_id=client_id, client_secret=client_secret, redirect_uri=redirect_uri)

sp = spotipy.Spotify(auth=token)
tracks = []

offset = 0
while True:
    track_set = sp.current_user_saved_tracks(limit=50, offset=offset)['items']
    if track_set:
        tracks += track_set
    else:
        break
    offset += 50
print("%d tracks fetched" % len(tracks))


try:
    with open('artist_names', 'rb') as file:
        artist_names = pickle.load(file)
except FileNotFoundError:
    artist_names = {}
    
artist_tracks = {}
all_artists = set()
for i in range(len(tracks)):
    tracks[i] = tracks[i]['track']
    track_id = tracks[i]['id']
    
    for artist in tracks[i]['artists']:
        if artist['id'] not in artist_tracks:
            artist_tracks[artist['id']] = [track_id]
        else:
            artist_tracks[artist['id']].append(track_id)
    
        all_artists.add(artist['id'])
        artist_names[artist['id']] = artist['name']
        
print("%d artists fetched" % len(artist_tracks))

3798 tracks fetched
2575 artists fetched


In [20]:
import networkx as nx

def chunks(n, x):
    return [x[i:i + n] for i in range(0, len(x), n)]

artist_genres = {}
for chunk in chunks(50, list(artist_tracks.keys())):
    for artist in sp.artists(chunk)['artists']:
        id = artist['id']
        artist_genres[id] = artist['genres']
print("Genres fetched")

try:
    G = nx.read_graphml('related_artists.graphml')
except FileNotFoundError:
    G = nx.Graph()

try:
    with open('fetched_artists', 'rb') as file:
        fetched_artists = pickle.load(file)
except FileNotFoundError:
    fetched_artists = set()
    
i = 0
not_found = 0
for artist_id in all_artists:
    if artist_id not in fetched_artists:
        related_list = sp.artist_related_artists(artist_id)['artists']
        not_found += 1
        fetched_artists.add(artist_id)

        for related in related_list:
            artist_names[related['id']] = related['name']
            G.add_edge(artist_id, related['id'])

            if related['id'] not in fetched_artists:
                sub_related_list = sp.artist_related_artists(related['id'])['artists']
                not_found += 1
                fetched_artists.add(related['id'])

                for sub_related in sub_related_list:
                    artist_names[sub_related['id']] = sub_related['name']
                    G.add_edge(related['id'], sub_related['id'])


    #             ssub_related_list = sp.artist_related_artists(sub_related['id'])['artists']
    #             for ssub_related in ssub_related_list:
    #                 artist_names[ssub_related['id']] = artist['name']
    #                 G.add_edge(sub_related['id'], ssub_related['id'])

    i += 1
    if i % 100 ==0:
        print("%d/%d artists processed, %d fetched" % (i, len(all_artists), not_found))
        
for artist in list(artist_tracks.keys()):
    if not G.has_node(artist):
        del artist_tracks[artist]
        all_artists.remove(artist)

unnamed_artists = []
for artist in G.nodes:
    if artist not in artist_names:
        unnamed_artists.append(artist)
        
print("%s unnamed artists found, fetching names..." % len(unnamed_artists))
for chunk in chunks(50, unnamed_artists):
    for artist in sp.artists(chunk)['artists']:
        artist_names[artist['id']] = artist['name']
        
nx.write_graphml(G, 'related_artists.graphml')
with open('fetched_artists', 'wb') as file:
    pickle.dump(fetched_artists, file)
with open('artist_names', 'wb') as file:
    pickle.dump(artist_names, file)

print("All artists processed")

Genres fetched
100/2575 artists processed, 0 fetched
200/2575 artists processed, 0 fetched
300/2575 artists processed, 0 fetched
400/2575 artists processed, 0 fetched
500/2575 artists processed, 0 fetched
600/2575 artists processed, 0 fetched
700/2575 artists processed, 0 fetched
800/2575 artists processed, 0 fetched
900/2575 artists processed, 0 fetched
1000/2575 artists processed, 0 fetched
1100/2575 artists processed, 0 fetched
1200/2575 artists processed, 0 fetched
1300/2575 artists processed, 0 fetched
1400/2575 artists processed, 0 fetched
1500/2575 artists processed, 0 fetched
1600/2575 artists processed, 0 fetched
1700/2575 artists processed, 0 fetched
1800/2575 artists processed, 0 fetched
1900/2575 artists processed, 0 fetched
2000/2575 artists processed, 0 fetched
2100/2575 artists processed, 0 fetched
2200/2575 artists processed, 0 fetched
2300/2575 artists processed, 0 fetched
2400/2575 artists processed, 0 fetched
2500/2575 artists processed, 0 fetched
2 unnamed artists f

In [24]:
degrees = list(G.nodes)
for i in range(len(degrees)):
    degree = 0
    artist = degrees[i]
    for neighbor in nx.neighbors(G, artist):
        if neighbor in artist_tracks:
            degree += len(artist_tracks[neighbor])
    degrees[i] = (artist, degree)

degrees.sort(key=lambda tup: tup[1], reverse=True)
degrees = [node for node in degrees if node[0] not in artist_tracks]
degrees = degrees[:100]
top_artists = [node[0] for node in degrees]

for chunk in chunks(50, top_artists):
    for artist in sp.artists(chunk)['artists']:
        id = artist['id']
        artist_genres[id] = artist['genres']

In [25]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
from multiprocessing import Pool
from functools import partial

print("Graph nodes:", nx.number_of_nodes(G))
print("Graph edges:", nx.number_of_edges(G))
print("Graph components:", nx.number_connected_components(G))

artists = list(artist_tracks.keys()) + top_artists
distance = np.zeros((len(artists), len(artists)))
start = datetime.now()
iters = 0

with Pool(3) as pool:
    paths = pool.map(partial(nx.single_source_shortest_path_length, G), artists)

i = 0
for lengths in paths:
    source = artists[i]
    for j in range(len(artists)):
        target = artists[j]
        if target in lengths:
            distance[i][j] = lengths[target]
        else:
            distance[i][j] = np.inf
    iters += 1
    i += 1
    
print(datetime.now() - start)

Graph nodes: 83079
Graph edges: 362226
Graph components: 7
0:07:04.137179


In [26]:
import sklearn.cluster
import sklearn.metrics

connectivity = nx.to_numpy_matrix(G, nodelist=artists)
similarity = np.exp(-distance / distance[ np.isfinite(distance) ].std())
# for n_clusters in range(2, 13):
# dist = np.copy(distance)
# max_value = dist[ np.isfinite(dist) ].max()
# print(max_value)
# for i in range(len(dist)):
#     for j in range(len(dist)):
#         if dist[i][j] == np.inf:
#             dist[i][j] = max_value * 2

#     clustering = sklearn.cluster.AgglomerativeClustering(n_clusters=n_clusters, affinity='precomputed', linkage='average', compute_full_tree=True).fit_predict(dist)
# clustering = sklearn.cluster.DBSCAN(eps=3, min_samples=1, metric='precomputed', n_jobs=-1).fit_predict(dist)
#     print(np.unique(clustering))
#     print("%d cluster silhouette:" % len(np.unique(clustering)), sklearn.metrics.silhouette_score(dist, clustering, metric='precomputed'))
# print(sklearn.metrics.silhouette_score(dist, clustering, metric='precomputed'))

clustering = sklearn.cluster.SpectralClustering(n_clusters=8, affinity='precomputed').fit_predict(similarity)
# clustering = sklearn.cluster.AgglomerativeClustering(n_clusters=8, affinity='precomputed', linkage='average', compute_full_tree=True).fit_predict(dist)
print(np.unique(clustering))
artist_cluster = {}
cluster_artists = {}
for i in range(len(artists)):
    artist_cluster[artists[i]] = clustering[i]
    if clustering[i] not in cluster_artists:
        cluster_artists[clustering[i]] = [artists[i]]
    else:
        cluster_artists[clustering[i]].append(artists[i])
for clus in cluster_artists:
    print(clus, len(cluster_artists[clus]))

c:\python37\lib\site-packages\sklearn\manifold\spectral_embedding_.py:234: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.warn("Graph is not fully connected, spectral embedding"


[0 1 2 3 4 5 6 7]
6 124
2 254
4 332
0 626
5 366
1 397
3 325
7 141


In [27]:
counts = {'cluster': [], 'genre': [], 'count': []}
for cluster in cluster_artists:
    genre_count = dict()
    total_tracks = 0
    for artist in cluster_artists[cluster]:
        if artist not in artist_tracks:
            continue
        for genre in artist_genres[artist]:
            if genre in genre_count:
                genre_count[genre] += len(artist_tracks[artist])
            else:
                genre_count[genre] = len(artist_tracks[artist])
        total_tracks += len(artist_tracks[artist])
    for genre, count in genre_count.items():
        counts['cluster'].append(cluster)
        counts['genre'].append(genre)
        counts['count'].append(count / total_tracks)

counts = pd.DataFrame(counts)
counts.sort_values('count', ascending=False, inplace=True)
original_counts = counts.copy()

clusters_to_name = set(counts['cluster'].unique())
playlists = {}
cluster_names = {}
while clusters_to_name:
    cluster = counts.iloc[0]['cluster']
    name = counts.iloc[0]['genre']
    playlists[name] = cluster
    cluster_names[cluster] = name
    counts = counts[ (counts['cluster']!=cluster) & (counts['genre']!=name) ]
    clusters_to_name.remove(cluster)
    
playlist_tracks = dict()
from collections import OrderedDict
for name in playlists:
    playlist_tracks[name] = []
    cluster = playlists[name]
    for artist in cluster_artists[cluster]:
        if artist in artist_tracks:
            playlist_tracks[name] += artist_tracks[artist]

In [33]:
'62R1EpHT7Y1OzPA9fVKKP9' in all_artists

False

In [34]:
top_clustered = {}
for degree in degrees:
    artist = degree[0]
    cluster = artist_cluster[artist]
    if cluster not in top_clustered:
        top_clustered[cluster] = [degree]
    else:
        top_clustered[cluster].append(degree)
        
for cluster in top_clustered:
    print(cluster_names[cluster])
    for degree in top_clustered[cluster][:10]:
        print("\t%s: %d" % (artist_names[degree[0]], degree[1]))

brostep
	Tima Dee: 268
	Knife Party: 193
	Micah Martin: 190
	Ricky Remedy: 179
	DJ Sliink: 175
	Riot Ten: 172
	Aryay: 161
	Kayzo: 160
	Teddy Tuxedo: 157
	Doctor P: 154
bass trap
	Hippie Sabotage: 236
	Madds: 192
	Quix: 164
	Duskus: 148
	Adventure Club: 136
	Electric Mantis: 133
	Alex Barthol: 126
	CRAY: 121
	Big Wild: 119
	MEMBA: 110
edm
	DyCy: 226
	Kill The Noise Remix: 204
	Thomas Gold: 192
	Syn Cole: 184
	Bougenvilla: 176
	Luciana: 174
	Cobra Effect: 172
	Dannic: 162
	Inpetto: 136
	Boris Smith: 135
indietronica
	HEDO: 201
	KRUE: 146
	Flume: 145
	Shlohmo: 140
	Jamie XX: 121
	House Dj: 116
	Luttrell: 113
	Oliver: 95
	Nosaj Thing: 91
progressive house
	Krewella: 159
	Aaron Smith: 155
	16 Bit Lolitas: 117
	Jaytech: 104
tech house
	Dosem: 139
	ANNA: 131
	Jody Wisternoff: 127
	Mihalis Safras: 94
rap
	Athletixx: 120


In [ ]:
token = util.prompt_for_user_token(username, 'playlist-modify-private', client_id=client_id, client_secret=client_secret, redirect_uri=redirect_uri)
sp = spotipy.Spotify(auth=token)

for name in playlist_tracks:
    id = sp.user_playlist_create(username, 'sortify '+name, public=False)['id']
    for chunk in chunks(100, playlist_tracks[name]):
        sp.user_playlist_add_tracks(username, id, chunk)

In [ ]:
import pprint
pp = pprint.PrettyPrinter(indent=4)
pp.pprint(playlists)